In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test  = x_test.reshape(-1, 784).astype("float32") / 255.0

print(x_train.shape, x_test.shape)

In [ ]:
latent_dim = 32
sgd = SGD(learning_rate=0.05, momentum=0.9)

encoder = models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(784,)),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(latent_dim, activation='relu')
])

decoder = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(latent_dim,)),
    layers.Dense(128, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(784, activation='sigmoid')
])

autoencoder = models.Sequential([encoder, decoder])

In [ ]:
autoencoder.compile(
    optimizer= sgd,
    loss='mse'
)

history = autoencoder.fit(
    x_train, x_train,
    epochs=40,
    batch_size=256,
    validation_data=(x_test, x_test),
    verbose=1
)

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.title('TensorFlow Autoencoder Training')
plt.show()

In [ ]:
n = 8
decoded_imgs = autoencoder.predict(x_test[:n])

plt.figure(figsize=(16,4))
for i in range(n):
    ax = plt.subplot(2, n, i+1)
    plt.imshow(x_test[i].reshape(28,28), cmap='gray')
    plt.axis('off')

    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(decoded_imgs[i].reshape(28,28), cmap='gray')
    plt.axis('off')

plt.suptitle("Original (Top) vs Reconstructed (Bottom)")
plt.show()

In [ ]:
Z_train = encoder.predict(x_train)
Z_test  = encoder.predict(x_test)

print(Z_train.shape, Z_test.shape)

In [ ]:
svm = SVC(kernel='rbf', C=10, gamma='scale')
svm.fit(Z_train, y_train)

In [ ]:
y_pred = svm.predict(Z_test)

print("TF Encoder + SVM Accuracy:",
      accuracy_score(y_test, y_pred))

print(classification_report(y_test, y_pred))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (TensorFlow Encoder + SVM)")
plt.show()